## Investigate high error rate
When we overfit a model with `sold_price_loss` close to 0, and save and reload the model, we observ a high prediction mean error rate. This notebook focuses on why this is the case.

First lets do a toy example and verify this we can reproduce the problem:

In [1]:
import os
import sys

root_dir = f'{os.getcwd()}/../../'
print(f'{os.path.abspath(root_dir)} is added to the sys path')
sys.path.append(root_dir)

/Users/majid/git/housing is added to the sys path


In [2]:
from datetime import datetime

import optuna
import json
from pathlib import Path
import shutil

import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tempfile import NamedTemporaryFile

2022-12-12 20:03:00.929895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from housing_model.modeling.naive_deep.model_trainer import KerasModelTrainer
from housing_model.evaluations.keras_model_evaluator import eval_model_on_tfds
from housing_model.modeling.naive_deep.model_trainer_test import get_overfit_loss, keras_models_are_almost_equal
from housing_model.training.trainer import train_job, ExperimentSpec

In [4]:
train_ds = tfds.load("tf_housing", split="201912").take(4).cache()
config_dir = f"{root_dir}/housing_model"
experiment_config_file = f"{config_dir}/experiment.json"
model_path = Path("/tmp/tmp_model")

print(model_path)

/tmp/tmp_model


2022-12-12 20:03:05.713739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
with open(experiment_config_file) as fin:
    experiment_config = ExperimentSpec.from_json(fin.read())

ex_cnt = experiment_config.training.batch_size
train_ds = train_ds.take(ex_cnt).cache()
test_ds = train_ds.take(ex_cnt).cache()

keras_model = KerasModelTrainer.build(experiment_config.modeling, model_path)

overfit_loss = get_overfit_loss(train_ds, keras_model, experiment_config.training)

learned_model = keras_model.keras_model
keras_model.save()

keras_model = KerasModelTrainer.load(model_path)
loaded_model = keras_model.keras_model

keras_models_are_almost_equal(learned_model, loaded_model)

predictor = keras_model.make_predictor()

metric = eval_model_on_tfds(test_ds, predictor)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/10000
1/1 [==============================] - ETA: 0s - loss: 131.4525 - model_loss: 3.6257 - model_1_loss: 127.8267WARNING:tensorflow:Early stopping conditioned on metric `val_sold_price_loss` which is not available. Available metrics are: loss,model_loss,model_1_loss,val_loss,val_model_loss,val_model_1_loss


Weight `w_price` got updated to 0.0.
Weight `w_bits` got updated to 1.0.
1/1 [==============================] - 7s 7s/step - loss: 131.4525 - model_loss: 3.6257 - model_1_loss: 127.8267 - val_loss: 128.8357 - val_model_loss: 1.0090 - val_model_1_loss: 127.8267
Epoch 2/10000
1/1 [==============================] - ETA: 0s - loss: 128.8357 - model_loss: 1.0090 - model_1_loss: 127.8267WARNING:tensorflow:Early stopping conditioned on metric `val_sold_price_loss` which is not available. Available metrics are: loss,model_loss,model_1_loss,val_loss,val_model_loss,val_model_1_loss


Weight `w_price` got updated to 0.30000000000000004.
Weight `w_bits` got updated to 0.7.
1/1 [==============================] - 0s 80ms/step - loss: 128.8357 - model_loss: 1.0090 - model_1_loss: 127.8267 - val_loss: 5.0546 - val_model_loss: 0.5586 - val_model_1_loss: 4.4960
Epoch 3/10000
1/1 [==============================] - ETA: 0s - loss: 5.0546 - model_loss: 0.5586 - model_1_loss: 4.4960WARNING:tensorflow:Early stopping conditioned on metric `val_sold_price_loss` which is not available. Available metrics are: loss,model_loss,model_1_loss,val_loss,val_model_loss,val_model_1_loss


...

Weight `w_price` got updated to 1.0.
Weight `w_bits` got updated to 1.2308248131348267e-32.
1/1 [==============================] - 0s 65ms/step - loss: 1.0041e-04 - model_loss: 1.0036e-04 - model_1_loss: 5.3353e-08 - val_loss: 9.8274e-05 - val_model_loss: 9.8222e-05 - val_model_1_loss: 5.2143e-08
Epoch 208/10000
Batch 0: loss is almost zero
1/1 [==============================] - ETA: 0s - loss: 9.8274e-05 - model_loss: 9.8222e-05 - model_1_loss: 5.2143e-08WARNING:tensorflow:Early stopping conditioned on metric `val_sold_price_loss` which is not available. Available metrics are: loss,model_loss,model_1_loss,val_loss,val_model_loss,val_model_1_loss


Weight `w_price` got updated to 1.0.
Weight `w_bits` got updated to 8.615773691943786e-33.
1/1 [==============================] - 0s 63ms/step - loss: 9.8274e-05 - model_loss: 9.8222e-05 - model_1_loss: 5.2143e-08 - val_loss: 9.6186e-05 - val_model_loss: 9.6135e-05 - val_model_1_loss: 5.1025e-08


INFO:tensorflow:Assets written to: /tmp/tmp_model/keras_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp_model/keras_model/assets


0it [00:00, ?it/s]/Users/majid/git/housing/venv/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['bedroom', 'washroom', 'house_type', 'parking', 'basement', 'seller_price'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
4it [00:00, 19.40it/s]


In [6]:
predictor = keras_model.make_predictor()

metric = eval_model_on_tfds(test_ds, predictor)

4it [00:00, 34.65it/s]


In [7]:
metric.value['mean']

0.00020125860227641868

Next we retrive a model with zero `sold_price_loss` and load it:

In [ ]:
exp_root = Path(root_dir) / "exp"
sudy_name = "exp_dese_depth_and_embd_size"
exp_path = exp_root / sudy_name

storage = f"sqlite:////{os.path.abspath(exp_path)}/{sudy_name}.db"

study = optuna.create_study(
    storage=storage,
    study_name=sudy_name,
    load_if_exists=True
)

In [ ]:
best_params = study.best_trial.params
print(best_params)

In [ ]:
with open(exp_path / "hyper-params-config.json") as f:
    hyper_param_configs = json.load(f)
name_format = hyper_param_configs["name"]
dir_name = name_format.format(**best_params)

model_path = None
for d in exp_path.iterdir():
    if dir_name in str(d):
        model_path = d

assert model_path is not None and model_path.exists(),\
    f"cannot find the overfitted model in {str(model_path)}"

keras_model = KerasModelTrainer.load(model_path)

now load the data set (please manually verify the `ml_num`)

In [ ]:
predictor = keras_model.make_predictor()
metrics = {}
for data_split in ('201906', '201911', '201912', '202001', '202002', '202003'):
    data = tfds.load('tf_housing', split=data_split)
    metrics[data_split] = eval_model_on_tfds(data.take(100), predictor)
    


Check the metrics of model on the train data set:

In [ ]:
def expected_loss(metric):
    pdf = metric.value["hist"]["pdf"]
    loss = 0
    for val, p in enumerate(pdf): 
        loss += p * (np.log(1 + 0.1 * val) ** 2)
    return loss

for data_split, metric in metrics.items():
    print(f"{data_split} -> {metric.value['med']}")
    print(f"expected_loss -> {expected_loss(metric)}")

Check the loss value:

In [ ]:
with open(model_path / "hist.json") as f:
    hist = json.load(f)

train_loss = hist['sold_price_loss'][-1]
print(f"train loss = {train_loss}")

print(f"expected mean error is = {np.e ** np.sqrt(train_loss)}")



Check the loss is 0:

In [ ]:
dataset_size = len(list(train_ds))
train_params = TrainParams(
    batch_size=dataset_size, epochs=1, learning_rate=1e-2,
    early_stopping=EarlyStoppingSetting(patience=500)
)
arc_params = keras_model.model_params.arc_params

keras_model.keras_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=train_params.learning_rate),
    loss={
        arc_params.price_feature_name: tf.keras.losses.MeanSquaredLogarithmicError(),
        arc_params.bits_feature_name: tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights={
        arc_params.price_feature_name: 0.99,
        arc_params.bits_feature_name: 0.01
    }
)

hist = keras_model.keras_model.fit(
    keras_model.data_provider.setup_data(train_ds, dataset_size),
    epochs=train_params.epochs
)

print(hist.history['loss'][-1])

The loss is not zero, was the model saved properly or the problem is in above code, first check if we can overfit the model again:

In [ ]:
hist = keras_model.fit_model(
    train_ds,
    None,
    # train_ds.take(dataset_size).cache(),
    TrainParams(
        batch_size=dataset_size, epochs=500, learning_rate=1e-2,
        early_stopping=EarlyStoppingSetting(patience=500)
    ),
)
print(hist.history['loss'][-1])

The model overfit agian, now we can check the above code without save/load the model again:

In [ ]:
hist = keras_model.keras_model.fit(
    keras_model.data_provider.setup_data(train_ds, dataset_size),
    epochs=train_params.epochs
)

print(hist.history['loss'][-1])

This code worked, please re-run above cell to make sure, everything is the same.

(I have done it and I got zero loss agian)

This suggest the problem is in loading and saving the model. Lets verify it:

In [ ]:
tmp_dir = Path('/tmp/tmp_model')
if tmp_dir.exists():
    shutil.rmtree(tmp_dir)
tmp_dir.mkdir()
    
keras_model.save(tmp_dir)
loaded_model = KerasModelTrainer.load(tmp_dir)

for lw, ow in zip(loaded_model.keras_model.weights, keras_model.keras_model.weights):
    print(lw - ow)


The model weight are the same!

In [ ]:
train_params = TrainParams(
    batch_size=dataset_size, epochs=1, learning_rate=1e-2,
    early_stopping=EarlyStoppingSetting(patience=500)
)
arc_params = loaded_model.model_params.arc_params

loaded_model.keras_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=train_params.learning_rate),
    loss={
        arc_params.price_feature_name: tf.keras.losses.MeanSquaredLogarithmicError(),
        arc_params.bits_feature_name: tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights={
        arc_params.price_feature_name: 0.99,
        arc_params.bits_feature_name: 0.01
    }
)
for lw, ow in zip(loaded_model.keras_model.weights, keras_model.keras_model.weights):
    print(lw - ow)



In [ ]:
res = eval_model_on_tfds(train_ds, keras_model.make_predictor())

In [ ]:
res.value

In [ ]:
for x, y in keras_model.data_provider.setup_data(train_ds, dataset_size):
    res = keras_model.keras_model(x)
    print(res['sold_price'].numpy())
    print(y['sold_price'].numpy())

In [ ]:
examples = [
    Example(ml_num=ex['metadata']['ml_num'].numpy().decode('UTF-8'), sold_price=int(ex[SOLD_PRICE].numpy().item()), features=Features(
        house_sigma_estimation=0.0,
        map_lat=ex[MAP_LAT].numpy().item(),
        map_lon=ex[MAP_LON].numpy().item(),
        land_front=ex[LAND_FRONT].numpy().item(),
        land_depth=ex[LAND_DEPTH].numpy().item(),
        date_end=datetime.fromtimestamp(
            int(ex[DATE_END].numpy().item() * 24 * 3600) 
            + datetime(1970, 1, 1).timestamp()
        )
    ))
    for ex in train_ds
]

encoded_features = [
    TfHousing.to_features(Example(sold_price=ex.sold_price, ml_num=ex.ml_num, features=ex.features))[1]
    for ex in examples
]

In [ ]:
for encoded in encoded_features:
    for original in train_ds:
        if encoded['metadata']['ml_num'] == original['metadata']['ml_num']:
            print(encoded)
            print()
            print(original)
            print()
            print("======")


In [ ]:
np.log(1.1) ** 2

In [ ]:
keras_model